In [1]:
import thermohubclient as thc
import thermofun as fun
import chemicalfun as cfun
import sys
sys.path.append('scripts-out')

In [2]:
from export import PhreeqcFormatter, secondary_master, master_formulas, master_symbols, reaction_properties, product_aqueous
from export import to_phreeqc_formation_reaction_prioritized, to_phreeqc_dissolution_reaction, process_reaction, correct_reactant_product_order, contains_solid, contains_gas, is_solid, is_liquid, is_gas
#from export import to_phreeqc_formation_reaction_prioritized, reaction_properties_json
from export import secondary_master_formulasx, secondary_masterx, master_formulasx, master_symbolsx, product_solids, zeolites

In [3]:
database = fun.Database('data-in/temperature/PsiTDB2020_with_tpmethods.json')
engine = fun.ThermoEngine(database)

### check for missing species

list1 = database.getReactionsList()
list2 = product_aqueous

for item in list1:
    if item not in list2:
        if is_aqeuous(database.getSubstance(item)):
            print(item)

### output a file to compare with existing db
with open("reaction_output_old.txt", "w") as file:
    file.close()

for r in product_aqueous:
    rr = database.getReaction(r)
    s = database.getSubstance(r)
    rcts = rr.reactants()
    reac_eq, sign = to_phreeqc_formation_reaction_prioritized(rr.symbol(),s.formula(),rcts)
    props = reaction_properties_json(rr.jsonString()) #reaction_properties(rr.thermoReferenceProperties())
    output = reaction_to_phreeqc_old(reac_eq, sign, props)
    # Append the result to a file
    with open("reaction_output_old.txt", "a") as file:
        file.write(output)  # Add newline for separation

# Export SECONDARY MASTER SPECIES 

In [4]:
# write them to file 
file_modes = { "data-out/phreeqc/psinagra2020_v2-0.txt": "w",
               "data-out/phreeqc/check/secondary_master.txt": "w" }

for path, mode in file_modes.items():
    with open(path, mode) as f:
        f.write("\n# SECONDARY MASTER SPECIES\n\n")
        f.close()

In [5]:
secondary_mastery = secondary_master #+ other_aqueous # adding other aqeuous without reactions to secondary master

secondary_master_species = {
    sm : database.getSubstance(sm)
    for sm in secondary_mastery
}

In [6]:
engine.setSolventSymbol('H2O(l)')
fmt = PhreeqcFormatter()
for symbol, s in secondary_master_species.items():
    formulas = master_formulas + [s.formula()]
    symbols  = master_symbols + [s.symbol()]
    output = process_reaction(s.symbol(), s.formula(), formulas, symbols,
                              engine, database, 
                              fmt.format_aqueous_reaction_generated)

    # write them to file 
    file_modes = { "data-out/phreeqc/psinagra2020_v2-0.txt": "a",
                   "data-out/phreeqc/check/secondary_master.txt": "a" }

    for path, mode in file_modes.items():
        with open(path, mode) as f:
            f.write(output + "\n")

# Export PRODUCT SPECIES

**(UO2)3O(OH)2HCO3+** fixed the name in the SIT list

In [7]:
reaction_entries = []
reactions_list_db = database.getReactionsList()

for r in product_aqueous:
    if r in reactions_list_db:
        rr = database.getReaction(r)
        s  = database.getSubstance(r)
        rcts = rr.reactants()

        if contains_solid(rcts, database) or contains_gas(rcts, database):
            # if reaction contains solid or gas in reactants, generat reactions 1st with secondary master species if it fails with primary master species
            output = process_reaction(
                s.symbol(), s.formula(),
                secondary_master_formulasx + master_formulasx + [s.formula()],
                secondary_masterx + master_symbolsx + [s.symbol()],
                engine, database,
                fmt.format_aqueous_reaction_generated,
                use_valence=True,
                retry_fallback=(master_formulas+[s.formula()],
                                master_symbols+[s.symbol()], False)
            )
            reactants = [] # list(rcts.keys())  # dict → list of symbols
        else:
            # take reaction as is in the TDB
            reac_eq, sign = to_phreeqc_formation_reaction_prioritized(rr.symbol(), s.formula(), rcts)
            props = reaction_properties(engine,rr)
            output = fmt.format_aqueous_reaction(rr, reac_eq, sign, props)
            reactants = list(rcts.keys())
    else:
        # if reaction not found in the tdb then generate it from master species
        s = database.getSubstance(r)
        output = process_reaction(
            s.symbol(), s.formula(),
            master_formulas + [s.formula()],
            master_symbols + [s.symbol()],
            engine, database,
            fmt.format_aqueous_reaction_generated
        )
        reactants = []  # or from reactions_dic[0]

    # clean insertion
    correct_reactant_product_order(reaction_entries, r, output, reactants)


In [8]:
# write them to file 
file_modes = { "data-out/phreeqc/psinagra2020_v2-0.txt": "a",
               "data-out/phreeqc/check/product_species.txt": "w" }

for path, mode in file_modes.items():
    with open(path, mode) as f:
        f.write("\n# PRODUCT SPECIES\n\n")
        f.close()

file_modes = { "data-out/phreeqc/psinagra2020_v2-0.txt": "a",
               "data-out/phreeqc/check/product_species.txt": "a" }

for path, mode in file_modes.items():
    with open(path, mode) as f:
        for entry in reaction_entries:
            # Each entry is a dict with one key: the species name
            species = list(entry.keys())[0]
            output = entry[species]["out"]
            f.write(output+'\n')  # Add newline for separation

In [9]:
substances = database.getSubstances()

solid_substances = {
    s.name().replace('_rdc_', ''): s
    for s in database.getSubstances()
    if is_solid(s) or is_liquid(s)
}
#print(solid_substances)

### Create a list of substances in product_solids not in solid_substances
missing_solids = [
    ps for ps in product_solids
    if ps not in solid_substances
]
print(missing_solids)

In [10]:
reactions_list = database.getReactionsList()
no_reactions_phases = [
    s.symbol()
    for s in database.getSubstances()
    if s.symbol() not in reactions_list and (is_solid(s) or is_liquid(s))
]

print(no_reactions_phases)

['Ag(cr)', 'Ag2Se(alpha)', 'Al(OH)3(cr)', 'Al2O3(alpha)', 'Analcime', 'Beidellite(Ca)', 'Beidellite(K)', 'Beidellite(Mg)', 'Beidellite(Na)', 'Beidellite_SBld-1', 'Berthierine(FeII)', 'Berthierine(FeIII)', 'Berthierine_ISGS', 'C3AS3(cr)', 'C3FS3(cr)', 'Ca(OH)2(cr)', 'CaCO3(calcite)', 'CaMg(CO3)2(cr)', 'CaSO4(cr)', 'CaSiO3(cr)', 'Cancrinite-NO3', 'Chabazite-Ca', 'Chabazite-Na', 'Clinoptilolite', 'Cronstedtite', 'Faujasite-X', 'Faujasite-Y', 'Fe(alpha)', 'Fe2O3(alpha)', 'Fe3O4(cr)', 'FeOOH(alpha)', 'FeOOH(gamma)', 'Glauconite', 'Heulandite_1', 'Heulandite_2', 'Hg(l)', 'Hg2Cl2(cr)', 'HgO(cr)', 'Hydrosodalite', 'Illite(Al)', 'Illite(FeII)', 'Illite(FeIII)', 'Illite(Mg)', 'Illite_IMt-2', 'Linda_type_A', 'Low-silica_P-Ca', 'Low-silica_P-Na', 'Mg(OH)2(s)', 'MgCO3(s)', 'Mn2O3(cr)', 'Mn3O4(cr)', 'MnCO3(cr)', 'MnO(cr)', 'MnO2(cr)', 'MoO2(cr)', 'MoO3(cr)', 'Molecular_sieve_4Å', 'Montmorillonite(HcCa)', 'Montmorillonite(HcK)', 'Montmorillonite(HcMg)', 'Montmorillonite(HcNa)', 'Montmorillonite(MgCa)

# Export PHASES GASES

In [11]:
# write them to file 
file_modes = { "data-out/phreeqc/psinagra2020_v2-0.txt": "a",
               "data-out/phreeqc/check/phases_gases.txt": "w" }

for path, mode in file_modes.items():
    with open(path, mode) as f:
        f.write("\nPHASES\n\n")
        f.write("# GASES\n\n")
        f.close()


In [12]:
gases = {
    s.symbol(): s
    for s in database.getSubstances()
    if is_gas(s)
}

In [13]:
master_formulas_g = ['C|-4|H4', 'C|+4|O2', 'H|0|2', 'H2S|-2|', 'H2Se|-2|', 'Hg|0|', 'N|0|2', 'O|0|2']
master_symbols_g = ['CH4(aq)', 'CO2(aq)', 'H2(aq)', 'H2S(aq)', 'H2Se(aq)', 'Hg(aq)', 'N2(aq)', 'O2(aq)']

for symbol, g in gases.items():
    formulas = [g.formula(), g.formula()]
    symbols  = [g.symbol().replace('(g)', '(aq)'), g.symbol()]

    output = process_reaction(
        g.symbol(), g.formula(),
        formulas, symbols,
        engine, database,
        fmt.format_phase_reaction_generated,
        mode="dissolution"  # tells helper to call to_phreeqc_dissolution_reaction
    )

    file_modes = { "data-out/phreeqc/psinagra2020_v2-0.txt": "a",
                   "data-out/phreeqc/check/phases_gases.txt": "a" }

    for path, mode in file_modes.items():
        with open(path, mode) as f:
            f.write(output + "\n")

### not working since the defines links for gases are removed.
for symbol, g in gases.items():
    if symbol not in no_reactions_phases:
        rr = database.getReaction(symbol)
        rcts = rr.reactants()
        reac_eq, sign = to_phreeqc_dissolution_reaction(symbol,g.formula(),rcts)
        props = reaction_properties_json(rr.jsonString()) #reaction_properties(rr.thermoReferenceProperties())
        output = phase_reaction_to_phreeqc(rr, g, reac_eq, sign, props)
        # Append the result to a file
        with open("psinagra2020_v2-0.txt", "a") as file:
            file.write(output)  # Add newline for separation

# Export PHASES MINERALS

In [14]:
# write them to file 
file_modes = { "data-out/phreeqc/psinagra2020_v2-0.txt": "a",
               "data-out/phreeqc/check/phases_minerals.txt": "w",
               "data-out/phreeqc/check/phases_zeolites.txt": "w" }

for path, mode in file_modes.items():
    with open(path, mode) as f:
        f.write("\n# MINERALS\n\n")
        f.close()

for r in product_solids:
    if r not in solid_substances:
        s = database.getSubstance(r)
    else:
        s = solid_substances[r]
    if s.symbol() not in no_reactions_phases:
        rr = database.getReaction(s.symbol())
        rcts = rr.reactants()
        if contains_gas(rcts, database) or contains_solid(rcts, database, s.symbol()):
            formulas = secondary_master_formulasx+master_formulasx+[s.formula()]
            symbols  = secondary_masterx+ master_symbolsx+[s.symbol()]
            output = process_reaction(
                s.symbol(), s.formula(),
                formulas, symbols,
                engine, database,
                fmt.format_phase_reaction_generated,
                use_valence=True,
                retry_fallback=(master_formulas+[s.formula()],
                                master_symbols+[s.symbol()], False),
                mode="dissolution"  # tells helper to call to_phreeqc_dissolution_reaction
            )
        else:
            # take reaction as is in the TDB
            reac_eq, sign = to_phreeqc_dissolution_reaction(rr.symbol(), s.formula(), rcts)
            props = reaction_properties(engine,rr)
            output = fmt.format_phase_reaction(rr, s, reac_eq, sign, props)
    else:
        # generate reaction and calculate its properties
        formulas = secondary_master_formulasx+master_formulasx+[s.formula()]
        symbols  = secondary_masterx+ master_symbolsx+[s.symbol()]
        output = process_reaction(
                s.symbol(), s.formula(),
                formulas, symbols,
                engine, database,
                fmt.format_phase_reaction_generated,
                use_valence=True,
                retry_fallback=(master_formulas+[s.formula()],
                                master_symbols+[s.symbol()], False),
                mode="dissolution"  # tells helper to call to_phreeqc_dissolution_reaction
            )
    # write them to file 
    file_modes = {
    "data-out/phreeqc/psinagra2020_v2-0.txt": "a",
    "data-out/phreeqc/check/phases_minerals.txt": "a",
    "data-out/phreeqc/check/phases_zeolites.txt": "a"
    }

    # Always write to phases_mineralsxx and phases_zeolitesxx
    for path in [
    "data-out/phreeqc/check/phases_minerals.txt",
    "data-out/phreeqc/check/phases_zeolites.txt"
    ]:
        with open(path, file_modes[path]) as f:
            f.write(output + "\n")

    # Write to psinagra2020_v2-0xx.txt only if not a zeolite
    if s.symbol() not in zeolites:
        path = "data-out/phreeqc/psinagra2020_v2-0.txt"
        with open(path, file_modes[path]) as f:
            f.write(output + "\n")
